# Pulling USCB ACS Data

This notebook pulls in ACS data at requested geo level and also has code for gcp bq load.

In [ ]:
import pandas as pd
import numpy as np
import json, requests, time

Change your parameters here:

In [2]:
## ACS Year of Data to pull
acs_year = 2019

## 5yr or 1yr estimates?  
acs_version = 5 

## Request API key from census bureau:
## https://api.census.gov/data/key_signup.html
census_key = 'input your key here'

## GEO LEVEL: ['TRACT', 'BLOCK GROUP', 'ZCTA']
geo_level = 'BLOCK GROUP'

In [3]:
## ACS COLUMNS TO GRAB 
## https://api.census.gov/data/2018/acs/acs5/variables.html


## [ ["ACS NAME", "PREFERRED NAME"] ]

acs_cols = [     
    ## Total Population
    ['B01003_001E', 'TOT_POP']
    
    ## Median Age 
    , ['B01002_001E', 'MED_AGE']
    
    ## MEDIAN HOUSEHOLD INCOME IN THE PAST 12 MONTHS
    , ['B19013_001E', 'MED_HH_INC']
]


***
***
***

In [4]:
if geo_level not in ('TRACT', 'BLOCK GROUP', 'ZCTA'):
    raise ValueError('Appropriate geo-level not defined')

In [5]:
acs_dicts={}
acs_df = pd.DataFrame()
decade = str(int(acs_year / 10) * 10)[2:4]

if geo_level == 'BLOCK GROUP':
    grp_cols = ['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE']
elif geo_level == 'TRACT':
    grp_cols = ['STATEFP', 'COUNTYFP', 'TRACTCE']
else:
    grp_cols = ['STATEFP', 'ZCTA5CE']


grp_cols = [s + str(decade) for s in grp_cols]


## LOOP THROUGH EACH COLUMN
for i1, y in enumerate(acs_cols):
    
    ## Grab states to loop through
    st_uscb_url = (
        ## ACS API DETAILED TABLES URL
        'https://api.census.gov/data/'
        +str(acs_year)+'/acs/acs'+str(acs_version)+'?get='
        ## ACS API VARIABLE
        +str(y[0])
        ## ALL STATES
        +'&for=state:*'
        ## CENSUS API KEY
        +'&key='+census_key
    )
    response = requests.get(st_uscb_url)
    data = json.loads(response.content.decode('utf-8'))
    
    if geo_level in ('ZCTA'):
        st_cds = ['*']
    else:
        st_cds = [item[1] for item in data]
        st_cds.remove('state')
    
 
    ## LOOP THROUGH EACH STATE
    for i, x in enumerate(st_cds):

        attempts = 0
        while attempts < 5:

            try:
                attempts += 1
                
                ## USCB API Examples: 
                    ## 1yr: https://api.census.gov/data/2019/acs/acs1/examples.html
                    ## 5yr: https://api.census.gov/data/2019/acs/acs5/examples.html
                scrape_uscb_url = (
                    ## ACS API DETAILED TABLES URL
                    'https://api.census.gov/data/'
                    ## ACS YEAR AND VERSION (1yr/5yr) OF DATA TO PULL
                    +str(acs_year)+'/acs/acs'+str(acs_version)+'?get='
                    ## ACS API VARIABLE
                    +str(y[0])
                    +'&for='
                    +( 'tract:*&in=state:'
                      if geo_level == 'TRACT'
                      else 'block%20group:*&in=county:*&in=state:'
                      if geo_level == 'BLOCK GROUP'                       
                      else 'zip%20code%20tabulation%20area:*&in=state:'
                     )
                    +str(x)
                    ## CENSUS API KEY
                    +'&key='+census_key
                )
                response = requests.get(scrape_uscb_url)
                data = json.loads(response.content.decode('utf-8'))

                headers = data.pop(0)
                headers[headers.index(y[0])] = y[1]
                headers[headers.index('state')] = 'STATEFP'+str(decade)
                if geo_level in ('BLOCK GROUP', 'TRACT'):
                    headers[headers.index('county')] = 'COUNTYFP'+str(decade)
                    headers[headers.index('tract')] = 'TRACTCE'+str(decade)
                    if geo_level == 'BLOCK GROUP':
                        headers[headers.index('block group')] = 'BLKGRPCE'+str(decade)
                else:
                    headers[headers.index('zip code tabulation area')] = 'ZCTA5CE'+str(decade)


                if y[0] in acs_dicts:
                    acs_dicts[y[0]] = acs_dicts[y[0]].append(
                        pd.DataFrame(data, columns=headers))                   
                else:
                    acs_dicts[y[0]] = pd.DataFrame(data, columns=headers)

            except Exception as e:
                #time.sleep(30*attempts)
                if attempts == 5:
                    ## YOU MAY WANT TO INVESTIGATE MISSING BLOCK GROUPS
                    print("Failure to pull after 5 attempts: "
                          +'state - '+str(x)+', column - '+str(y))
                    print("Error: "+str(e))
                    pass
            else:
                break

    
    if y[0] in acs_dicts:  
        
        acs_dicts[y[0]][y[1]] = pd.to_numeric(acs_dicts[y[0]][y[1]], errors='coerce').replace(
            ## https://www.census.gov/data/developers/data-sets/acs-1year/ -->
            ## notes-on-acs-estimate-and-annotation-values.html 
            [-999999999, -888888888, -666666666
             , -555555555, -333333333, -222222222]
            , np.nan)
        
        if acs_df.empty:     
            acs_df = acs_dicts[y[0]] 
            acs_df.insert(len(acs_df.columns)-1, y[1], acs_df.pop(y[1]))
        else:  
            acs_df = pd.merge(
                    acs_df
                    , acs_dicts[y[0]]
                    , on=grp_cols
                    , how = 'outer'
                )


In [6]:
acs_df.dtypes

STATEFP10      object
COUNTYFP10     object
TRACTCE10      object
BLKGRPCE10     object
TOT_POP         int64
MED_AGE       float64
MED_HH_INC    float64
dtype: object

In [7]:
acs_df.count()

STATEFP10     220333
COUNTYFP10    220333
TRACTCE10     220333
BLKGRPCE10    220333
TOT_POP       220333
MED_AGE       219143
MED_HH_INC    212034
dtype: int64

In [8]:
acs_df.min()

STATEFP10         01
COUNTYFP10       001
TRACTCE10     000100
BLKGRPCE10         0
TOT_POP            0
MED_AGE          5.9
MED_HH_INC    2499.0
dtype: object

In [9]:
acs_df.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLKGRPCE10,TOT_POP,MED_AGE,MED_HH_INC
0,01,039,962000,2,884,40.8,28250.0
1,01,039,961800,2,1395,58.0,22817.0
2,01,039,961600,4,793,44.4,48088.0
3,01,039,961600,2,942,35.1,64432.0
4,01,039,961600,1,1726,36.3,53081.0


***
***
***

### Load Dataframe into Google BigQuery

In [10]:
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField, DatasetReference

In [13]:
## GCP Billing Project
bq_bill_prj = "your-billing-project"

## GCP BQ DESTINATION TABLE 
bq_prj = 'destination-prj'
bq_ds = 'destination_dataset'
bq_tbl = 'destination_table'

In [14]:
# Function to upload pandas dataframe into a non streaming table in BigQuery
def df_to_bq_tbl(proj,ds,tbl,df):
    
    bq_client = bigquery.Client(bq_prj)

    dataset_ref = DatasetReference(proj, ds)

    job_config = bigquery.LoadJobConfig()
    job_config.autodetect =True
    job_config.write_disposition='WRITE_TRUNCATE'

    load_job = bq_client.load_table_from_dataframe(df,dataset_ref.table(tbl),job_config=job_config) 
    load_job.result()
    destination_table = bq_client.get_table(dataset_ref.table(tbl))

In [ ]:
df_to_bq_tbl(bq_prj, bq_ds, bq_tbl, acs_df)